<a href="https://colab.research.google.com/github/mohamed-stifi/PFA-Arabic-LLMs/blob/main/prompting_and_proceeding_mawdoo3_data_to_queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def filter_fun(tag):
  """
    Custom filter function to be used with BeautifulSoup find_all.
    Filters out script tag.

    Parameters:
    - tag (bs4.element.Tag): A BeautifulSoup Tag object.

    Returns:
    - bool: True if the tag should be included, False otherwise.
  """
  return tag.name != "script"

def del_attrs(soup):
    """
    Recursively removes all attributes from the given BeautifulSoup tree.

    Parameters:
    - soup (bs4.BeautifulSoup): A BeautifulSoup object representing the HTML document.
    """
    tags = soup.find_all(True, recursive= False)
    if len(tags) > 0:
        for tag in tags:
            tag.attrs = {}
            del_attrs(tag)


In [ ]:
def mowdoo3_topics(url = 'https://mawdoo3.com'):
    """
    Scrapes and retrieves a list of topic URLs from the Mawdoo3 website.

    Parameters:

    """

    # Send a GET request to the specified URL
    res = requests.get(url)

    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(res.content, 'html.parser')

    # Find the main section containing the topics
    main_section = soup.find_all('section', class_ = 'container home')[0]

    # Find the block content within the main section
    block_content = main_section.find_all('div', class_ = "row block-content")[0]

    # Find all 'a' tags within the block content
    a_tags = block_content.find_all('a')
    topics = []
    # Extract href attributes from 'a' tags and construct full URLs
    for tag in a_tags:
        topic_title = tag['href'].split(':')[1].replace("_"," ")
        topic_url = 'https://mawdoo3.com'+tag['href']
        val = (topic_title, topic_url)
        topics.append(val)
    return topics

In [ ]:
def articles_of_mawdoo3_topic(topic_url):
        """
        Scrapes and retrieves a list of article URLs for a specific topic on Mawdoo3.

        Parameters:
        - topic_url (str): The URL of the Mawdoo3 topic page.

        Returns:
        - list: A list of strings representing the full URLs of articles in the specified topic.
        """

        # Send a GET request to the specified topic URL
        res = requests.get(topic_url)

        # Parse the HTML content of the topic page using BeautifulSoup
        soup = BeautifulSoup(res.content, 'html.parser')

        # Find the content section containing the articles
        content = soup.find_all('div', class_ = 'columns large-8 medium-12 small-12')[0].find_all('ul', id = "grid")[0]

        # Find all 'a' tags within the content section
        a_tags = content.find_all('a')

        # Extract href attributes from 'a' tags and construct full URLs
        articls_urls = ['https://mawdoo3.com'+tag['href'] for tag in a_tags]

        # Pagination loop
        i = 2
        while len(a_tags) == 132:

            # Generate the URL for the next page
            next_page = topic_url+f'?page={i}'

            # Send a GET request to the next page
            res = requests.get(next_page)
            soup = BeautifulSoup(res.content, 'html.parser')

            # Find the content section on the next page
            content = soup.find_all('div', class_ = 'columns large-8 medium-12 small-12')[0].find_all('ul', id = "grid")[0]
            # Find all 'a' tags on the next page
            a_tags = content.find_all('a')

            # Break the loop if no more articles are found
            if len(a_tags) == 0:
                break

            # Extract href attributes from 'a' tags on the next page and add to the list
            for tag in a_tags:
                articls_urls.append('https://mawdoo3.com'+tag['href'])

            i = i + 1
        return articls_urls

In [ ]:
def div_article_to_paragraphs(article_html):
    parg = article_html.split("<h2>")
    n = len(parg)
    content = []
    for i in range(1, n):
        all_html = parg[i].split('</h2>')
        soup = BeautifulSoup(all_html[0], 'html.parser')
        paragraph_title = soup.get_text(strip=True)

        if  "المراجع" != paragraph_title and "المراجع." != paragraph_title and "المراجعنص غليظ" != paragraph_title and 'فيديو' not in paragraph_title:
            soup = BeautifulSoup(all_html[1], 'html.parser')
            text = soup.get_text(strip=True)
            paragraph_text = text.replace(",", "\n").replace('\t', '').replace('.', '\n').strip()
            if i == (n-1) :
                if "\n" not in paragraph_text and len(paragraph_text) < 99:
                  break


            content.append({paragraph_title:paragraph_text})

    return content

In [ ]:
import json
path1 = "/content/drive/MyDrive/mowdoo3 _dataset/articls_of_mawdoo3_from_1_to_22748.json"
path2  = "/content/drive/MyDrive/mowdoo3 _dataset/articls_of_mawdoo3_from_49765_to_77714.json"
with open(path1, "r", encoding="utf-8") as json_file:
    articles1 = json.load(json_file)
with open(path2, "r", encoding="utf-8") as json_file:
    articles2= json.load(json_file)

In [ ]:
# Concatenate the two lists
articles = articles1['data'] + articles2['data']
len(articles), len(articles[0])

(77712, 4)

In [ ]:
first_elements = [sublist[3] for sublist in articles]
len(first_elements), len(set(first_elements))

(77712, 67788)

In [ ]:
import pandas as pd
columns = ['article_titel', 'article_text', 'article_html', 'article_url']
df_art = pd.DataFrame(data = articles, columns = columns)
df_art.head()

,article_titel,article_text,article_html,article_url
0,طريقة عمل البسبوسة,محتويات\n١\nطريقة البسبوسة التقليدية\n١.١\nالم...,<h2><span><b>طريقة البسبوسة التقليدية</b></spa...,https://mawdoo3.com/طريقة_عمل_البسبوسة
1,طريقة عمل شوربة العدس,محتويات\n١\nشوربة العدس\n١.١\nالمكونات\n١.٢\nط...,"<h2><span><b> شوربة العدس</b></span></h2>, <p>...",https://mawdoo3.com/طريقة_عمل_شوربة_العدس
2,طريقة عمل عجينة البيتزا,محتويات\n١\nالبيتزا\n٢\nعجينة البيتزا\n٢.١\nال...,"<h2><span><b>البيتزا</b></span></h2>, <p>هناك ...",https://mawdoo3.com/طريقة_عمل_عجينة_البيتزا
3,طريقة عمل البشاميل,محتويات\n١\nالبشاميل\n١.١\nالمكوّنات\n١.٢\nطري...,"<h2><span><b>البشاميل</b></span></h2>, <pre>يع...",https://mawdoo3.com/طريقة_عمل_البشاميل
4,كيفية عمل السحلب,محتويات\n١\nالسّحلب بنكهة ماء الزهر\n١.١\nالمك...,<h2><span><b>السّحلب بنكهة ماء الزهر</b></span...,https://mawdoo3.com/كيفية_عمل_السحلب


In [ ]:
topics = mowdoo3_topics()
articles_count = 1
errors = []
data = []  # [{topic_title: [{article_title : [{paragraphe_title : paragraphe_text}]}]}]

In [ ]:
for topic_id, (topic_title, topic_url) in enumerate(topics) :
    data.append({topic_title : []})
    articles_url = articles_of_mawdoo3_topic(topic_url)

    for article_id, article_url in enumerate(articles_url):
        article_title = article_url.replace('https://mawdoo3.com/', '').replace("_", ' ')

        # [{topic_title: [{article_title : []}]}]
        # if articles_count == 352: print(data[topic_id])

        data[topic_id][topic_title].append({article_title:[]})

        matching_articles = df_art[df_art['article_titel'] == article_title].reset_index(drop=True)

        if len(matching_articles) > 0:
            article_html = matching_articles.head(1)['article_html'][0]

            # article_paragraphs = [{paragraphe_title : paragraphe_text}]
            article_paragraphs = div_article_to_paragraphs(article_html)

            # [{topic_title: [{article_title : [{paragraphe_title : paragraphe_text}]}]}]
            data[topic_id][topic_title][article_id][article_title] = article_paragraphs

        # print('---------------------------------------------: ', articles_count)
        # articles_count += 1

In [ ]:
import json
path = '/content/drive/MyDrive/mowdoo3 _dataset/full_database.json'
with open(path, "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

In [ ]:
len(data) # number of topics

357

In [ ]:
import json
path = '/content/drive/MyDrive/mowdoo3 _dataset/full_database.json'

# Load the article content from the JSON file
with open(path, "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

# prompting


# data = [{topic_title: [{article_title : [{paragraphe_title : paragraphe_text}]}]}]

In [ ]:
def generate_prompt(topic_title, article_title, paragraph_title, paragraph_text):
    return f"""يوجد أدناه تعليمات تصف مهمة، إلى جانب إدخال يوفر المزيد من السياق. اكتب ردًا يكمل الطلب بشكل مناسب.
    ### تعليمات:
    أكتب فقرة حول موضوع {topic_title}, في حدود {article_title} و المدخل التالي
    ### مدخل:
    {paragraph_title}
    ### انتاج:
    {paragraph_text}"""

In [ ]:
import re
def generate_prompt_of_data(data):
    querys = []
    # Define the pattern to match Arabic numerals inside square brackets
    pattern = r'\[\d+\]'
    for ind, topic in enumerate(data):
        topic_title , topic_articls = list(topic.items())[0]
        for article in topic_articls:
            article_title, article_paragraphs = list(article.items())[0]
            for paragraph in article_paragraphs:
                paragraph_title, paragraph_text = list(paragraph.items())[0]

                if  "المراجع" != paragraph_title and "المراجع." != paragraph_title and "المراجعنص غليظ" != paragraph_title and 'فيديو' not in paragraph_title and len(paragraph_text) > 99:
                    # Use re.sub() to replace matches with an empty string
                    paragraph_text = re.sub(pattern, ' ', paragraph_text)
                    prompt = generate_prompt(topic_title, article_title, paragraph_title, paragraph_text)

                    querys.append(prompt)
        # if ind%15 == 0: print("topic : ", ind)

    return set(querys)

In [ ]:
queries = list(generate_prompt_of_data(data))

In [ ]:
path = '/content/drive/MyDrive/mowdoo3 _dataset/list_of_queries.json'
with open(path, "w", encoding="utf-8") as json_file:
    json.dump(queries, json_file, ensure_ascii=False, indent=4)

In [ ]:
len(queries)    # 260597

259642

In [ ]:
print(queries[207564])   # 207564

يوجد أدناه تعليمات تصف مهمة، إلى جانب إدخال يوفر المزيد من السياق. اكتب ردًا يكمل الطلب بشكل مناسب.
    ### تعليمات:
    أكتب فقرة حول موضوع فيتامينات ومعادن, في حدود أسماء فيتامينات لتقوية الذاكرة و المدخل التالي
    ### مدخل:
    عناصر أخرى مفيدة للذاكرة
    ### انتاج:
    أوميغا 3:تُعدّ أحماض أوميغا 3 الدهنيّة ومكمّلات زيت السمك من العناصر الغذائية التي تؤثر في الذاكرة، حيثُ بيّنت العديد من الدراسات أنّ زيادة تناول الأطعمة التي تحتوي على هذاالأحماض الدهنيةكالزيوت النباتية، وزيوت المكسرات، وأسماك المياه الباردة، والجوز الإنجليزي، قد تُساهم في تقليل خطر الإصابة بمرض ألزهايمر بشكل ملحوظ،كما أظهرت دراسةٌ أُجريت على الفئران ونُشرت في مجلة PLOS ONE عام 2012 أنّ اتّباع نظامٍ غذائي يحتوي على على كميّة جيّدة من أحماض أوميغا 3 الدهنيّة قد يُساعد على تقليل خطر إصابة الدماغ بالالتهابات العصبية والضعف الإدراكي المُرتبط بالشيخوخة

وللاطلاع على مزيد من المعلومات حول فوائد اوميغا 3 يمكن قراءة مقالما فوائد اوميغا 3

الزنك:فقد يُساهم عنصر الزنك في تنظيم عملية التواصل بينالخلايا العصبية، ممّا يؤثر في 

# **get topics and article title**

In [ ]:
def generate_prompt_of_data(data):
    topics = [['topic_id', 'topic_title']]    # [topic_id, topic_title]
    articles = [['article_id', 'article_title', 'topic_id']]      # [article_id, article_title, topic_id]
    a_id = 1
    for t_id, topic in enumerate(data):
        topic_title , topic_articls = list(topic.items())[0]

        topics.append([t_id+1, topic_title])
        for article in topic_articls:
            article_title, article_paragraphs = list(article.items())[0]

            articles.append([a_id, article_title, t_id+1])
            a_id += 1

    return topics, articles

In [ ]:
topics, articles = generate_prompt_of_data(data)

In [ ]:
import pandas as pd

In [ ]:
df_topics = pd.DataFrame(data = topics[1:], columns=topics[0])
df_articles = pd.DataFrame(data = articles[1:], columns=articles[0])

In [ ]:
df_topics.head()

,topic_id,topic_title
0,1,فن الطهي
1,2,أطباق رئيسية
2,3,أطباق جانبية
3,4,أطباق شرقية
4,5,أطباق شامية


In [ ]:
df_articles.head()

,article_id,article_title,topic_id
0,1,طريقة عمل البسبوسة,1
1,2,طريقة عمل شوربة العدس,1
2,3,طريقة عمل عجينة البيتزا,1
3,4,طريقة عمل البشاميل,1
4,5,كيفية عمل السحلب,1


In [ ]:
path = '/content/drive/MyDrive/PFA/dataset/'
df_articles.to_csv(path + 'article_table.csv' , index=False)
df_topics.to_csv(path + 'topic_table.csv', index=False)